In [1]:
import findspark
findspark.init()

from pyspark.mllib.recommendation import ALS, Rating
from pyspark import SparkContext, SQLContext
sc = SparkContext("local", "test")
sqlContext = SQLContext(sc)

In [2]:
TRAIN_FILE = "./debug_data/ratings-train-debug.dat/"
VALIDATION_FILE = "./debug_data/ratings-validation-debug.dat"
TEST_FILE = "./debug_data/ratings-test-debug.dat/"

In [4]:
def prepare_data(data):
    return (
        data
        .map(lambda l: l.split('::'))
        .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    )  

In [5]:
# Load and parse the data
ratings_train_text = sc.textFile(TRAIN_FILE)
ratings_train = prepare_data(ratings_train_text)
ratings_train.take(10)

[Rating(user=45211, product=2126, rating=4.0),
 Rating(user=45186, product=8, rating=3.0),
 Rating(user=45186, product=805, rating=5.0),
 Rating(user=45186, product=1003, rating=4.0),
 Rating(user=43733, product=3994, rating=2.0),
 Rating(user=45186, product=230, rating=4.0),
 Rating(user=45211, product=2253, rating=5.0),
 Rating(user=45186, product=367, rating=5.0),
 Rating(user=45211, product=3439, rating=5.0),
 Rating(user=45186, product=1918, rating=5.0)]

In [6]:
ratings_validation_text = sc.textFile(VALIDATION_FILE)
ratings_validation = prepare_data(ratings_validation_text)
ratings_validation.take(10)

[Rating(user=67269, product=6944, rating=4.0),
 Rating(user=67123, product=914, rating=4.0),
 Rating(user=67269, product=1101, rating=4.0),
 Rating(user=67258, product=1500, rating=0.5),
 Rating(user=67258, product=6708, rating=3.5),
 Rating(user=67123, product=3256, rating=3.5),
 Rating(user=67123, product=1918, rating=3.0),
 Rating(user=67269, product=5168, rating=5.0),
 Rating(user=67123, product=1562, rating=2.5),
 Rating(user=67123, product=1962, rating=4.0)]

In [7]:
ratings_test_text = sc.textFile(TEST_FILE)
ratings_test = prepare_data(ratings_validation_text)
ratings_test.take(10)

[Rating(user=67269, product=6944, rating=4.0),
 Rating(user=67123, product=914, rating=4.0),
 Rating(user=67269, product=1101, rating=4.0),
 Rating(user=67258, product=1500, rating=0.5),
 Rating(user=67258, product=6708, rating=3.5),
 Rating(user=67123, product=3256, rating=3.5),
 Rating(user=67123, product=1918, rating=3.0),
 Rating(user=67269, product=5168, rating=5.0),
 Rating(user=67123, product=1562, rating=2.5),
 Rating(user=67123, product=1962, rating=4.0)]

#### Calculate the general mean u for all ratings

In [8]:
global_mean = ratings_train.map(lambda r: (r[2])).mean()

In [9]:
global_mean

4.090000000000001

##### calculate item-specific bias, according to the paper we referenced, for each item i, its bias is equal to the summation of difference between all ratings of to the same item and global mean and then the result is divided by the sum of a regulation parameter and the quantity of the ratings.


In [10]:
#convert training data to dataframe with attribute
df = sqlContext.createDataFrame(ratings_train, ['userId', 'movieId', 'ratings'])

In [11]:
#sort the data by movie
df_orderByMovie = df.orderBy(df.movieId)

In [12]:
#group the movie and count each movie
movie_count = df_orderByMovie.groupBy(df_orderByMovie.movieId).count()

In [13]:
#calculate the sum of the ratings of each movie
sum_byMovie = df_orderByMovie.groupBy(['movieId']).sum()

In [14]:
#drop some unrelated column
drop_column1 = sum_byMovie.drop(sum_byMovie[1])
final_drop = drop_column1.drop(drop_column1[1])

In [15]:
#join the sum of count and sum of rating for each movie
movie_sorted = movie_count.join(final_drop, "movieId")

In [16]:
#sorted the dataset by each movie
new_movie_sorted = movie_sorted.orderBy(movie_sorted.movieId)

In [17]:
#calculate item specific bias
item_bias = new_movie_sorted.map(lambda r: [r[0], (r[2] - r[1]*global_mean)/(25+r[1])])

In [18]:
new_item_bias = sqlContext.createDataFrame(item_bias, ['movieId', 'item_bias'])

#### Caculate the user-specific bias

In [19]:
#order the training set by user
df_orderByUser = df.orderBy(df.userId)

In [20]:
#join the item bias dataset to with the same movieId
contain_itemBias = df_orderByUser.join(new_item_bias, "movieId")

In [21]:
#sorted the dataset by user
sorted_byUser = contain_itemBias.orderBy(['userId'])

In [22]:
#calculate the numerical part of item specific bais
subtraction = sorted_byUser.map(lambda r: [r[1], r[2] - global_mean - r[3]])

In [23]:
user_bias_part1 = sqlContext.createDataFrame(subtraction, ['userId', 'subtraction'])

In [24]:
sum_byUser = user_bias_part1.groupBy(['userId']).sum()

In [25]:
#count the user 
sum_UserCollect = user_bias_part1.groupBy(['userId']).count()

In [26]:
#order the data set by user
ordered_sum_UserCollect = sum_UserCollect.orderBy(sum_UserCollect.userId)

In [27]:
drop_column2 = sum_byUser.drop(sum_byUser[1])

In [28]:
final_drop2 = drop_column2.orderBy(drop_column2.userId)

In [29]:
user_bias_table = final_drop2.join(ordered_sum_UserCollect, 'userId')

In [30]:
ordered_userBiaTable = user_bias_table.orderBy(user_bias_table.userId)

In [31]:
user_bias = ordered_userBiaTable.map(lambda r: [r[0], r[1]/(10+r[2])])

In [32]:
user_specific_bias = sqlContext.createDataFrame(user_bias, ['userId', 'user_bias'])

In [33]:
merge1 = df_orderByUser.join(user_specific_bias, 'userId')

In [34]:
merge2 = merge1.join(new_item_bias, 'movieId')

In [35]:
new_ratings_train = merge2.map(lambda r: [r[0], r[1], r[2] - r[3] - r[4]])

In [36]:
temp = sqlContext.createDataFrame(new_ratings_train, ['movieId', 'userId', 'new_ratings'])

In [37]:
final_new_ratings_train = temp.orderBy(temp.userId)

In [38]:
final_new_ratings_train.take(10)

[Row(movieId=479, userId=33700, new_ratings=4.017884615384615),
 Row(movieId=3409, userId=33700, new_ratings=4.017884615384615),
 Row(movieId=1032, userId=43733, new_ratings=4.128919413919414),
 Row(movieId=3638, userId=43733, new_ratings=5.090457875457876),
 Row(movieId=2078, userId=43733, new_ratings=3.167380952380953),
 Row(movieId=2081, userId=43733, new_ratings=2.2058424908424916),
 Row(movieId=2881, userId=43733, new_ratings=5.093663003663004),
 Row(movieId=2085, userId=43733, new_ratings=3.167380952380953),
 Row(movieId=2087, userId=43733, new_ratings=4.128919413919414),
 Row(movieId=2687, userId=43733, new_ratings=3.167380952380953)]

In [39]:
#now, we perform the same procedure as task1
#first, we sort the data by timestamp. 
new_ratings_byTime = final_new_ratings_train.join(df, ['userId', 'movieId'])

In [40]:
#example of dataset
new_ratings_byTime.take(20)

[Row(userId=45211, movieId=2542, new_ratings=4.393715574548908, ratings=5.0),
 Row(userId=43733, movieId=1032, new_ratings=4.128919413919414, ratings=3.0),
 Row(userId=45186, movieId=74, new_ratings=4.008408584169454, ratings=4.0),
 Row(userId=43733, movieId=3638, new_ratings=5.090457875457876, ratings=4.0),
 Row(userId=45211, movieId=153, new_ratings=4.393715574548908, ratings=5.0),
 Row(userId=45186, movieId=2881, new_ratings=4.011613712374583, ratings=4.0),
 Row(userId=45211, movieId=2959, new_ratings=4.393715574548908, ratings=5.0),
 Row(userId=45186, movieId=2485, new_ratings=4.008408584169454, ratings=4.0),
 Row(userId=45211, movieId=2762, new_ratings=4.393715574548908, ratings=5.0),
 Row(userId=45186, movieId=292, new_ratings=4.008408584169454, ratings=4.0),
 Row(userId=45186, movieId=2094, new_ratings=4.008408584169454, ratings=4.0),
 Row(userId=45186, movieId=2496, new_ratings=3.0468701226309927, ratings=3.0),
 Row(userId=33700, movieId=479, new_ratings=4.017884615384615, rati

In [41]:
new_ratings_byTime = new_ratings_byTime.drop(new_ratings_byTime[3])

In [66]:
new_ratings_byTime.take(10)

[Row(userId=45211, movieId=2542, new_ratings=4.393715574548908),
 Row(userId=43733, movieId=1032, new_ratings=4.128919413919414),
 Row(userId=45186, movieId=74, new_ratings=4.008408584169454),
 Row(userId=43733, movieId=3638, new_ratings=5.090457875457876),
 Row(userId=45211, movieId=153, new_ratings=4.393715574548908),
 Row(userId=45186, movieId=2881, new_ratings=4.011613712374583),
 Row(userId=45211, movieId=2959, new_ratings=4.393715574548908),
 Row(userId=45186, movieId=2485, new_ratings=4.008408584169454),
 Row(userId=45211, movieId=2762, new_ratings=4.393715574548908),
 Row(userId=45186, movieId=292, new_ratings=4.008408584169454)]

In [72]:
type(new_ratings_byTime)

pyspark.sql.dataframe.DataFrame

In [42]:
def prepare_validation(validation):
    return validation.map(lambda p: (p[0], p[1]))

In [43]:
import math

In [45]:
print type(new_ratings_byTime)
print type(ratings_validation)

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.rdd.PipelinedRDD'>


In [49]:
temp_validation = sqlContext.createDataFrame(ratings_validation, ['userId', 'movieId', 'new_ratings'])

In [67]:
temp_validation.take(10)

[Row(userId=67269, movieId=6944, new_ratings=4.0),
 Row(userId=67123, movieId=914, new_ratings=4.0),
 Row(userId=67269, movieId=1101, new_ratings=4.0),
 Row(userId=67258, movieId=1500, new_ratings=0.5),
 Row(userId=67258, movieId=6708, new_ratings=3.5),
 Row(userId=67123, movieId=3256, new_ratings=3.5),
 Row(userId=67123, movieId=1918, new_ratings=3.0),
 Row(userId=67269, movieId=5168, new_ratings=5.0),
 Row(userId=67123, movieId=1562, new_ratings=2.5),
 Row(userId=67123, movieId=1962, new_ratings=4.0)]

In [50]:
type(temp_validation)

pyspark.sql.dataframe.DataFrame

In [68]:
# Evaluate the model on training data

def train_evaluate_als(train, validation, rank, iterations_num, lambda_val):
#     print("Validation %s" % validation.take(100))
    model = ALS.train(train, rank, iterations_num, lambda_val)
    model.
    predictions = model.predictAll(prepare_validation(validation)).map(lambda r: ((r[0], r[1]), r[2]))
    print("Predictions %s" % model.predictAll(prepare_validation(validation)))
    ratesAndPreds = validation.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
#     print("ratesAndPreds %s" % ratesAndPreds.take(100))
    MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
    RMSE = math.sqrt(MSE)
    return MSE, RMSE

In [69]:
ranks = [10, 20, 30, 40, 50]
lambda_values = [0.01,0.1,1.0,10.0]
ITERATIONS = 10

In [70]:
def report_mse_results(rank, lambda_value, mse, rmse):
    print("Rank=%d, Lambda=%0.2f, MSE=%s, RMSE=%s" % (rank, lambda_value, mse, rmse))

In [71]:
def evaluate_parameters(train, validation, ranks, lambda_values):
    for r in ranks:
        for l in lambda_values:
            mse, rmse = train_evaluate_als(train, validation, r, ITERATIONS, l)
            report_mse_results(r, l, mse, rmse)

In [73]:
evaluate_parameters(new_ratings_byTime, temp_validation, ranks, lambda_values)

Predictions MapPartitionsRDD[1651] at mapPartitions at PythonMLLibAPI.scala:1522
Rank=10, Lambda=0.01, MSE=0.0, RMSE=0.0
Predictions MapPartitionsRDD[1896] at mapPartitions at PythonMLLibAPI.scala:1522
Rank=10, Lambda=0.10, MSE=0.0, RMSE=0.0


KeyboardInterrupt: 